In [110]:
import json
import pandas as pd
import numpy as np

In [111]:
file_dir = '/Users/ilan/git/movies-etl/'

In [112]:
with open(f'{file_dir}/wikipedia-movies.json',mode='r') as file:
    wiki_movies_raw = json.load(file)

In [113]:
kaggle_metadata = pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory=False)
ratings = pd.read_csv(f'{file_dir}ratings.csv')

In [114]:
# Check the data
kaggle_metadata.sample(n=5)
ratings.tail()

,userId,movieId,rating,timestamp
26024284,270896,58559,5.0,1257031564
26024285,270896,60069,5.0,1257032032
26024286,270896,63082,4.5,1257031764
26024287,270896,64957,4.5,1257033990
26024288,270896,71878,2.0,1257031858


In [115]:
# Check for issues with an immediate import
wiki_movies_df = pd.DataFrame(wiki_movies_raw)

# too many columns

In [116]:
# Pull out only movies with a Director or Directed by and an IMDB link
wiki_movies = [movie for movie in wiki_movies_raw 
               if('Director' in movie or 'Directed by' in movie)
               and 'imdb_link' in movie
               and 'No. of episodes' not in movie]
wiki_movies_df = pd.DataFrame(wiki_movies)

#some rows and columns removed
sorted(wiki_movies_df.columns.tolist())



['Adaptation by',
 'Also known as',
 'Animation by',
 'Arabic',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cantonese',
 'Chinese',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'French',
 'Genre',
 'Hangul',
 'Hebrew',
 'Hepburn',
 'Japanese',
 'Label',
 'Language',
 'Length',
 'Literally',
 'Mandarin',
 'McCune–Reischauer',
 'Music by',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Original release',
 'Original title',
 'Picture format',
 'Polish',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Revised Romanization',
 'Romanized',
 'Running time',
 'Russian',
 'Screen story by',
 'Screenplay by',
 'Simplifie

In [117]:
def clean_movie(movie):
    movie = dict(movie)  #Create a non-destructive copy

    alt_titles = {}
    for language in ['Arabic', 'Cantonese', 'Chinese', 'French', 'Hangul', 'Hebrew', 'Hepburn', 'Japanese', 
                     'Literally', 'Mandarin', 'McCune–Reischauer', 'Original title', 'Polish',  'Revised Romanization', 'Romanized', 'Russian', 'Simplified', 'Traditional', 'Yiddish']:
        if language in movie:
            alt_titles[language] = movie[language]
            movie.pop(language)
    if len(alt_titles) > 0:
        movie["alt_titles"] = alt_titles
    return movie
    
    

In [118]:
clean_movies = [clean_movie(movie) for movie in wiki_movies]

In [119]:
wiki_movies_df = pd.DataFrame(clean_movies)